# Problem 12

The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28. The first ten terms would be:

1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...

Let us list the factors of the first seven triangle numbers:

```
 1: 1
 3: 1,3
 6: 1,2,3,6
10: 1,2,5,10
15: 1,3,5,15
21: 1,3,7,21
28: 1,2,4,7,14,28
```

We can see that 28 is the first triangle number to have over five divisors.

What is the value of the first triangle number to have over five hundred divisors?

In [5]:
def triangle(n: int) -> int:
    return int((n + 1) * n / 2)

In [11]:
# https://stackoverflow.com/a/3941967/62997
# https://gist.github.com/mbarkhau/afad5ea4d3640d58df50251b449e49d7
def mk_primes(limit):
    a = [True] * limit                          # Initialize the primality list
    a[0] = a[1] = False

    for i, isprime in enumerate(a):
        if isprime:
            yield i
            for n in range(i*i, limit, i):     # Mark factors non-prime
                a[n] = False

In [17]:
prime_list = list(mk_primes(100))
prime_list = list(mk_primes(10000))
prime_set = set(prime_list)

In [18]:
divisor_dict = {} 
def divisors0(n):
    '''this is probably wrong -- are you sure you want to use this?'''    
    ds = divisor_dict.get(n)
    if ds:
        return ds 
    
    ds = {1, n}
    
    if n in prime_set:
        return ds
    
    upto = n ** 0.5
    
    for p in prime_list:
        if p > upto:
            break
        if n % p == 0:
            ds = ds | { p } |  divisors0( n // p)
            
    divisor_dict[n] = ds 
    return ds

In [26]:
for n in range(1, 100000):
    # t = triangle(n)
    t = (n * (n + 1)) // 2
    divs1 = divisors0(n)
    divs2 = divisors0(n + 1)
    if len(divs1) * len(divs2) < 600:
        continue
    divs = divisors0(t)
    print(f"{n:>5} : {t:>5} {len(divs)}")
    if len(divs) > 500:
        break

12375 : 76576500 576


In [27]:
for n in range(1, 100000):
    t = triangle(n)
    divs = divisors0(t)
    if len(divs) > 500:
        print(f"{n:>5} : {t:>5} {len(divs)}")
        break

12375 : 76576500 576


# Problem 73 - Counting fractions in a range


Consider the fraction, n/d, where n and d are positive integers. If n<d and HCF(n,d)=1, it is called a reduced proper fraction.

If we list the set of reduced proper fractions for d ≤ 8 in ascending order of size, we get:

```
1/8, 1/7, 1/6, 1/5, 1/4, 2/7, 1/3,   3/8, 2/5, 3/7,   1/2, 4/7, 3/5, 5/8, 2/3, 5/7, 3/4, 4/5, 5/6, 6/7, 7/8
```

It can be seen that there are 3 fractions between 1/3 and 1/2.

How many fractions lie between 1/3 and 1/2 in the sorted set of reduced proper fractions for d ≤ 12,000?

In [57]:
import functools

# @functools.lru_cache(100000)
def gcd(a, b):
    """Returns the greatest common divisor of a and b.
    Should be implemented using recursion.

    >>> gcd(34, 19)
    1
    >>> gcd(39, 91)
    13
    >>> gcd(20, 30)
    10
    >>> gcd(40, 40)
    40
    """
    if b > a:
        return gcd(b, a)

    if a % b == 0:
        return b

    return gcd(b, a % b)     

In [58]:
def iter_fractions(n):
    lo = 1/3
    hi = 1/2
    for i in range(1, n + 1):
        for j in range(1, i):
            if gcd(i, j) > 1:
                continue
            f = j / i
            if lo < f < hi:
                yield (f, f"{j}/{i}")

In [59]:
len(sorted(iter_fractions(12000)))

7295372

# Problem 102 - Triangle containment


In [76]:
import collections

Point = collections.namedtuple("Point", ["x", "y"])

def parse_triangles():
    with open("p102_triangles.txt") as fh:
        for line in fh:
            ax, ay, bx, by, cx, cy = map(int, line.strip().split(","))
            a = Point(ax, ay)
            b = Point(bx, by)
            c = Point(cx, cy)
            yield a, b, c
            
triangles = list(parse_triangles())

In [77]:
triangles[0]

(Point(x=-340, y=495), Point(x=-153, y=-910), Point(x=835, y=-947))

In [78]:
o = Point(0, 0)

def nv(a, b):
    """ Computes a normal vector to the vector created of the points a and b. 
        Unique up to scaling
    """
    return Point(b.y - a.y, (-1)*(b.x - a.x))
    
def nf(a, b, p):
    n = nv(a, b)
    scalar_product = n.x * (p.x - a.x) + n.y * (p.y - a.y)
    return scalar_product

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

In [85]:
triangle_origin_interior = []
for A, B, C in triangles:
    # line AB
    ab_1 = sign(nf(A, B, o))
    ab_2 = sign(nf(A, B, C)) 
    if ab_1 != ab_2:
        continue
        
    # line BC
    bc_1 = sign(nf(B, C, o))
    bc_2 = sign(nf(B, C, A)) 
    if bc_1 != bc_2:
        continue
        
    # line AC
    ac_1 = sign(nf(A, C, o))
    ac_2 = sign(nf(A, C, B)) 
    if ac_1 != ac_2:
        continue
    
    triangle_origin_interior += [(A, B, C)]
    
print("Number of triangles with origin in the interior: ", len(triangle_origin_interior))

Number of triangles with origin in the interior:  228


In [84]:
triangle_origin_interior[1]

(Point(x=419, y=-864), Point(x=-83, y=650), Point(x=-399, y=171))

# Problem 68 - Magic 5-gon ring

attempt to find linear algebra solution

```
matrix

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |   
|---|---|---|---|---|---|---|---|---|---|----|
| a | 1 | 1 | 1 |   |   |   |   |   |   |    |   
| b |   |   | 1 | 1 | 1 |   |   |   |   |    |   
| c |   |   |   |   | 1 | 1 | 1 |   |   |    |   
| d |   |   |   |   |   |   | 1 | 1 | 1 |    |   
| e |   | 1 |   |   |   |   |   | 1 |   |  1 |   

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |   
|---|---|---|---|---|---|---|---|---|---|----|
| a | 1 | 1 | 1 |   |   |   |   |   |   |    |   
| e |   | 1 |   |   |   |   |   | 1 |   |  1 |   
| c |   |   |   |   | 1 | 1 | 1 |   |   |    |   
| d |   |   |   |   |   |   | 1 | 1 | 1 |    |   
| b |   |   | 1 | 1 | 1 |   |   |   |   |    |   

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |   
|---|---|---|---|---|---|---|---|---|---|----|
| a | 1 | 1 | 1 |   |   |   |   |   |   |    |   
| e |   | 1 |   |   |   |   |   | 1 |   |  1 |   
| b |   |   | 1 | 1 | 1 |   |   |   |   |    |   
| d |   |   |   |   |   |   | 1 | 1 | 1 |    |   
| c |   |   |   |   | 1 | 1 | 1 |   |   |    |   

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |   |
|---|---|---|---|---|---|---|---|---|---|----|---|
| a | 1 | 1 | 1 |   |   |   |   |   |   |    | s |
| e |   | 1 |   |   |   |   |   | 1 |   |  1 | s |
| b |   |   | 1 | 1 | 1 |   |   |   |   |    | s |
| c |   |   |   |   | 1 | 1 | 1 |   |   |    | s |
| d |   |   |   |   |   |   | 1 | 1 | 1 |    | s |

a - e

|   | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8  | 9 | 10 |   |
|---|---|---|---|---|---|---|---|----|---|----|---|
| a | 1 | 0 | 1 |   |   |   |   | -1 |   | -1 | 0 |
| e |   | 1 |   |   |   |   |   |  1 |   |  1 | s |
| b |   |   | 1 | 1 | 1 |   |   |    |   |    | s |
| c |   |   |   |   | 1 | 1 | 1 |    |   |    | s |
| d |   |   |   |   |   |   | 1 |  1 | 1 |    | s |

a - b

|   | 1 | 2 | 3 | 4  | 5  | 6 | 7 | 8  | 9 | 10 |    |
|---|---|---|---|----|----|---|---|----|---|----|----|
| a | 1 | 0 | 0 | -1 | -1 |   |   | -1 |   | -1 | -s |
| e |   | 1 |   |    |    |   |   |  1 |   |  1 | s  |
| b |   |   | 1 |  1 |  1 |   |   |    |   |    | s  |
| c |   |   |   |    |  1 | 1 | 1 |    |   |    | s  |
| d |   |   |   |    |    |   | 1 |  1 | 1 |    | s  |

a + c

|   | 1 | 2 | 3 | 4  | 5 | 6 | 7 | 8  | 9 | 10 |   |
|---|---|---|---|----|---|---|---|----|---|----|---|
| a | 1 | 0 | 0 | -1 | 0 |   |   | -1 |   | -1 | 0 |
| e |   | 1 |   |    |   |   |   |  1 |   |  1 | s |
| b |   |   | 1 |  1 | 1 |   |   |    |   |    | s |
| c |   |   |   |    | 1 | 1 | 1 |    |   |    | s |
| d |   |   |   |    |   |   | 1 |  1 | 1 |    | s |

b - c 

|   | 1 | 2 | 3 | 4  | 5 | 6  | 7  | 8  | 9 | 10 |   |
|---|---|---|---|----|---|----|----|----|---|----|---|
| a | 1 | 0 | 0 | -1 | 0 |    |    | -1 |   | -1 | 0 |
| e |   | 1 |   |    |   |    |    |  1 |   |  1 | s |
| b |   |   | 1 |  1 | 0 | -1 | -1 |    |   |    | 0 |
| c |   |   |   |    | 1 |  1 |  1 |    |   |    | s |
| d |   |   |   |    |   |    |  1 |  1 | 1 |    | s |

a - d

|   | 1 | 2 | 3 | 4  | 5 | 6  | 7  | 8  | 9  | 10 |    |
|---|---|---|---|----|---|----|----|----|----|----|----|
| a | 1 | 0 | 0 | -1 | 0 |    |    | -2 | -1 | -1 | -s |
| e |   | 1 |   |    |   |    |    |  1 |    |  1 | s  |
| b |   |   | 1 |  1 | 0 | -1 | -1 |    |    |    | 0  |
| c |   |   |   |    | 1 |  1 |  1 |    |    |    | s  |
| d |   |   |   |    |   |    |  1 |  1 |  1 |    | s  |

b + d

|   | 1 | 2 | 3 | 4  | 5 | 6  | 7 | 8  | 9  | 10 |    |
|---|---|---|---|----|---|----|---|----|----|----|----|
| a | 1 | 0 | 0 | -1 | 0 |    |   | -2 | -1 | -1 | -s |
| e |   | 1 |   |    |   |    |   |  1 |    |  1 | s  |
| b |   |   | 1 |  1 | 0 | -1 | 0 |  1 |  1 |    | s  |
| c |   |   |   |    | 1 |  1 | 1 |    |    |    | s  |
| d |   |   |   |    |   |    | 1 |  1 |  1 |    | s  |

c - d 

|   | 1 | 2 | 3 | 4  | 5 | 6  | 7 | 8  | 9  | 10 |    |
|---|---|---|---|----|---|----|---|----|----|----|----|
| a | 1 | 0 | 0 | -1 | 0 |    |   | -2 | -1 | -1 | -s |
| e |   | 1 |   |    |   |    |   |  1 |    |  1 | s  |
| b |   |   | 1 |  1 | 0 | -1 | 0 |  1 |  1 |    | s  |
| c |   |   |   |    | 1 |  1 | 0 | -1 | -1 |    | 0  |
| d |   |   |   |    |   |    | 1 |  1 |  1 |    | s  |

```

In [108]:
import itertools

def iter_rings(nums, group_coords):
    for perm in itertools.permutations(nums):
        prev_s = None
        is_group = True
        for coords in group_coords:
            s = sum((perm[c] for c in coords))
            if prev_s and prev_s != s:
                is_group = False
                break
            prev_s = s

        if not is_group:
            continue

        groups = [
            [perm[c] for c in coords]
            for coords in group_coords
        ]
        if min(groups) != groups[0]:
            continue
        ring = "".join(map(str, sum(groups, [])))
        # print(groups, ring)
        yield ring

In [109]:
nums_5 = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
nums_3 = [6, 5, 4, 3, 2, 1]

group_coords_5 = [
    [0, 5, 6],
    [1, 6, 7],
    [2, 7, 8],
    [3, 8, 9],
    [4, 9, 5],
]

group_coords_3 = [
    [0, 3, 4],
    [1, 4, 5],
    [2, 5, 3],
]

In [110]:
max(iter_rings(nums_3, group_coords_3))

'432621513'

In [111]:
max(iter_rings(nums_5, group_coords_5))

'6531031914842725'

In [112]:
sum(map(int, "653 1031 914 842 725"))

ValueError: invalid literal for int() with base 10: ' '

In [113]:
len("6531031914842725")

16